In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import stan
import arviz as az

import nest_asyncio
nest_asyncio.apply()

## Introduction

In this notebook, we will examine a simulated task focused on **word production**. The real task involves a study of individuals with **aphasia** — a language disorder often resulting from a stroke or head injury. 

In our simulated task, participants are shown an image, such as that of a cat, and asked to identify it. Responses can vary, for instance, a participant might:

1. correctly identify the image as a *cat*,
2. produce a non-word that has a phonological relation to the target (i.e., a **neologism**), such as *cag*,
3. provide a semantically unrelated, yet phonologically similar word like *hat* (i.e., a **formal** mistake),
4. offer a semantically and phonologically similar but incorrect identification such as *rat* (i.e., a **mixed** mistake), or
5. omit the response altogether or provide a response that does not fit the above categories (e.g., a lengthy description)

Below, we we build a theoretical multiprocessing tree model (MPT) to explain the frequencies of these outcomes.

In [ ]:
# Define a local RNG with a fixed seed
global_rng = np.random.default_rng(42)

# Fix true probabilities for simulation study
true_probs = {
    'correct': .42,
    'neologism': .1,
    'formal': .2,
    'mixed': .08,
    'other': .2 # 1 - rest
}


# Simulate data
num_trials = 100
y = global_rng.multinomial(n=num_trials, pvals=list(true_probs.values()))
pd.DataFrame(y[:, None].T, columns=true_probs.keys(), index=['responses'])

,correct,neologism,formal,mixed,other
responses,47,12,19,4,18


## Modeling Multiple Responses via a Multinomial Likelihood

We will first build a baseline model that will estimate the underlying (latent) probabilities from the observed frequency data.

### Stan Code

In [3]:
baseline_program_code = """
data {
  int<lower=1> N; // Number of trials
  int<lower=1> K; // Number of categories
  array[K] int<lower=0, upper=N> freqs;
}

parameters {
  // Our code here
}

model {
 // Our code here
}

generated quantities{
  // Obtaining predictions
  array[K] int preds = multinomial_rng(theta, N);
}
"""

### Model Fitting

In [ ]:
# Prepare data
stan_dict = {
    'freqs': y,
    'N': num_trials,
    'K': y.shape[0]
}

# Compile model
posterior = stan.build(baseline_program_code, data=stan_dict, random_seed=42)


# Sample (i.e., inverse inference)
fit = posterior.sample(num_chains=4, num_samples=2500, num_warmup=1000)

### Model Inspection

In [ ]:
# Estimation summary, convergence, and efficiency diagnostics
az.summary(fit)

In [ ]:
# Fix true probabilities for simulation study
true_probs = {
    'correct': .42,
    'neologism': .1,
    'formal': .2,
    'mixed': .08,
    'other': .2 
}

In [ ]:
# Traceplots and marginals - visual convergence checks
axarr = az.plot_trace(fit)
plt.tight_layout()

In [ ]:
# Inspect all outputs
fit.to_frame().head()

## Modeling Multiple Responses via a Multinomial Processing Tree (MPT) Model.

We will now build a process model that will estimate the underlying (latent) **process probabilities** from the observed frequency data.

Walker, Hickok, and Fridriksson (2018) created an MPT model that specifies a set of possible internal errors that lead to the various possible response types during a picture naming trial for aphasic patients. Here we’ll explore a simplification of the original model.

Our model assumes that when an attempt is made to produce a word, errors in production can arise at the whole word level (lexical level) or the morpheme level (phonological level). Semantic errors are assumed to arise from the lexical substitutions, and neologism errors from phonological substitutions. Real word responses that are phonologically related to the correct target word can arise from substitutions at the lexical or phonological level.

1. Walker, G. M., Hickok, G., & Fridriksson, J. (2018). A cognitive psychometric model for assessment of picture naming abilities in aphasia. *Psychological Assessment, 30*(6), 809. http://dx.doi.org/10.1037/pas0000529 

### Parameters and Model

The model is depicted in the following figure:

<img src="mpt_fig.png" width=65% height=65% />


The table below lists the four latent parameters and corresponding interpretations:


| Parameter | Description | 
| --- | --- |
| $a$ | Probability of initiating an attempt |
| $t$ | Probability of selecting a target word from a pool of candidates |
| $f$ | Probability of retrieving correct phonemes |
| $c$ | Probability of a phoneme change in the target word |

The table below lists the five types of responses (categories):


| Response (Category) | Description | Example | 
| --- | --- | --- |
| Correct | The response matches the target image. | cat |
| Neologism | The response is not a word, but it has a phonological relation to the target image. | cag |
| Formal | The response is a word with only a phonological relation to the target image. | hat |
| Mixed | The response is a word with both a semantic and phonological relation the target image. | rat |
| Other | All other responses, including omissions, descriptions, non-nouns, etc. | - |


### Simulating the Model

Practice: We first need to derive the equations for the category probabilities according to the graphical model.

In [ ]:
# Fix true parameters
true_params = {
    'a': 0.75,
    't': 0.9,
    'f': 0.8,
    'c': .2
}
a = true_params["a"]
t = true_params["t"]
f = true_params["f"]
c = true_params["c"]

# Fix true probabilities and simulate from model
num_trials = 120
true_probs = {
    'correct': a * t * f,
    'neologism': a * (1 - t) * (1 - f) * (1 - c) + a * t * (1 - f) * (1 - c),
    'formal': a * (1 - t) * (1 - f) * c + a * t * (1 - f) * c,
    'mixed': a * (1 - t) * f,
    'other': 1 - a
}

data = global_rng.multinomial(n=num_trials, pvals=list(true_probs.values()))

In [ ]:
mpt_model_code = """
data {
  int<lower=1> N; // Number of trials
  int<lower=1> K; // Number of categories
  array[K] int<lower=0, upper=N> freqs;
}

parameters {
  // Your code here
}

transformed parameters {
  simplex[K] theta;
  // Your code here - need to fill in the entries of the simplex according to the tree
}

model {
  // Assuming uniform priors
  target += beta_lpdf(a | 1, 1);
  target += beta_lpdf(t | 1, 1);
  target += beta_lpdf(f | 1, 1);
  target += beta_lpdf(c | 1, 1);

  // Multinomial likelihood
  target += multinomial_lpmf(freqs | theta);
}

generated quantities{
  array[K] int pred_freqs;
  pred_freqs = multinomial_rng(theta, N);
}
"""

In [3]:
### Compile, fit, and diagnose model
### Your code here

In [4]:
### Parameter recovery and predictions

## Global Parameter Recovery Study

In [1]:
### Our code here